In [1]:
import nbimporter
import math
import numpy

# 暖冷房設備の能力等の設定

## 対流式暖冷房（ルームエアコンディショナー）

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
2 エネルギー消費性能の算定方法  
2.1 算定方法  
第四章 暖冷房設備  
第七節 温水暖房  
付録A 機器の性能を表す仕様の決定方法   

A.2 定格能力  
式(1)～(2) 
ただし、定格冷房能力$q_{rtd,C}$が 5600(W)を超える場合は、定格冷房能力$q_{rtd,C}$は 5600(W)に等しいものとする。

$$
\displaystyle
q_{rtd,C}= 190.5 \times A_{HCZ} + 45.6
$$

$$
\displaystyle
q_{rtd,H}= 1.2090 \times q_{rtd,C} + 85.1
$$

$A_{HCZ}$: ルームエアコンディショナーにより暖冷房される暖冷房区画の床面積, m<sup>2</sup>  
$e_{rtd,C}$: 定格冷房能力, W  
$e_{rtd,H}$: 定格暖房能力, W  

A.3 最大能力  
式(3)～(4)   
ただし、定格暖房能力$q_{rtd,C}$が最大暖房能力$q_{max,C}$を超える場合は、最大暖房能力$q_{max,C}$は定格暖
房能力$q_{rtd,C}$に等しいとする。  
ただし、定格暖房能力$q_{rtd,H}$が最大暖房能力$q_{max,H}$を超える場合は、最大暖房能力$q_{max,H}$は定格暖
房能力$q_{rtd,H}$に等しいとする。

$$
\displaystyle
q_{max,C}= 0.8462 \times q_{rtd,C} + 1205.9
$$

$$
\displaystyle
q_{max,H}= 1.7597 \times q_{max,C} - 413.7
$$

$q_{max,H}$: 最大冷房能力, W  
$q_{max,C}$: 最大暖房能力, W  

A.4 定格エネルギー消費効率  
式(5)～(6)  

$$
e_{rtd,C} = \left \{
\begin{array}{ll}
-0.533 \times 10^{-3} \times e_{rtd,C} + 6.34 & ( 区分（い） ) \\
-0.504 \times 10^{-3} \times e_{rtd,C} + 5.88 & ( 区分（ろ） ) \\
-0.473 \times 10^{-3} \times e_{rtd,C} + 5.50 & ( 区分（は） )
\end{array}
\right.
$$

$$
\displaystyle
e_{rtd,H}= 0.77 \times e_{rtd,C} + 1.66
$$

$e_{rtd,H}$: 定格暖房エネルギー消費効率  
$e_{rtd,C}$: 定格冷房エネルギー消費効率  

#### Function

In [26]:
def get_capacity_RAC(A):
    
    q_rtd_c = min(5600, 190.5*A + 45.6)
                
    q_rtd_h = 1.2090*q_rtd_c - 85.1   
        
    q_max_c = max(q_rtd_c, 0.8462*q_rtd_c + 1205.9)
    
    q_max_h = max(q_rtd_h, 1.7597*q_max_c - 413.7)
        
    return {'q_rtd_c': q_rtd_c, 'q_rtd_h': q_rtd_h, 'q_max_c': q_max_c, 'q_max_h': q_max_h}

In [10]:
def get_efficiency_RAC(q_rtd_c, division):
    
    d_coefficient = {
        '（い）': { 'a': -0.000553, 'b': 6.34 },
        '（ろ）': { 'a': -0.000504, 'b': 5.88 },
        '（は）': { 'a': -0.000473, 'b': 5.50 }       
    }
    
    e_rtd_c = d_coefficient[division]['a']*q_rtd_c + d_coefficient[division]['b']

    e_rtd_h = 0.77*e_rtd_c + 1.66

    return {'e_rtd_c': e_rtd_c, 'e_rtd_h': e_rtd_h}

#### Example

In [12]:
d = {
    'Amain'  : 30.0,
    'Aother' : 60.0,
    'Dmain'  : '（い）',
    'Dother' : '（ろ）'
}

q_main = get_capacity_RAC(d['Amain'])
e_main = get_efficiency_RAC(q_main['q_rtd_c'], d['Dmain'])

q_other = get_capacity_RAC(d['Aother'])
e_other = get_efficiency_RAC(q_other['q_rtd_c'], d['Dother'])

print(q_main,e_other,q_other,e_other)

{'q_rtd_c': 5600, 'q_rtd_heating': 6685.3, 'q_max_c': 5944.619999999999, 'q_max_h': 10047.047813999998} {'e_rtd_c': 3.0576, 'e_rtd_h': 4.014352} {'q_rtd_c': 5600, 'q_rtd_heating': 6685.3, 'q_max_c': 5944.619999999999, 'q_max_h': 10047.047813999998} {'e_rtd_c': 3.0576, 'e_rtd_h': 4.014352}


## 放射式暖冷房（温水床暖房）

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
2 エネルギー消費性能の算定方法  
2.1 算定方法  
第四章 暖冷房設備  
第七節 温水暖房  
付録L 温水床暖房   
L.5 敷設面積   
式(4)

$$
\displaystyle
A_{f}= A_{HCZ} \times A_{Af}
$$

$A_{f}$: 温水床暖房の敷設面積, m<sup>2</sup>  
$A_{HCZ}$: 暖冷房区画の床面積, m<sup>2</sup>  
$r_{Af}$: 温水床暖房の敷設率  

#### Function

In [7]:
def get_construct_area_FH(A, r):
    
    return A*r

#### Example

In [8]:
A = 30
r = 0.5

get_construct_area_FH(30, 0.5)

15.0

## 暖冷房設備種類より必要情報を取得

暖冷房設備の種類が対流式の場合は、ルームエアコンディショナーの定格能力・最大能力・定格能力を取得する。  
暖房設備が放射式の場合は、床暖房の敷設面積を取得する。

#### Function

In [31]:
def convert(A, cooling, heating):
    
    d = {
        'cooling':{'main':{}, 'sub' :{}},
        'heating':{'main':{}, 'sub' :{}}        
    }

    # 冷房
    # 対流式
    if cooling['type'] == 'convection' :
        q_RAC_c = get_capacity_RAC(A)
        e_RAC_c = get_efficiency_RAC(q_RAC_c['q_rtd_c'], cooling['division'])
        d['cooling']['main'] = {
            'type'   : 'room_air_conditioner',
            'q_rtd_c': q_RAC_c['q_rtd_c'], 
            'q_max_c': q_RAC_c['q_max_c'],
            'e_rtd_c': e_RAC_c['e_rtd_c'],
        }        
    elif cooling['type'] == 'radiation' :
        raise ValueError
    else :
        raise ValueError   

    # 暖房
    # 対流式
    if heating['type'] == 'convection' :
        q_RAC_h = get_capacity_RAC(A)
        e_RAC_h = get_efficiency_RAC(q_RAC_h['q_rtd_c'], cooling['division'])
        d['heating']['main'] = {
            'type'   : 'room_air_conditioner',
            'q_rtd_h': q_RAC_h['q_rtd_h'], 
            'q_max_h': q_RAC_h['q_max_h'],
            'e_rtd_h': e_RAC_h['e_rtd_h'],
        }        
    # 放射式
    elif heating['type'] == 'radiation' :
        d['heating']['main'] = {
            'type'   : 'floor_heating',
            'construct_area': get_construct_area_FH(A, heating['r']),
        }         
        q_RAC_h = get_capacity_RAC(A)
        e_RAC_h = get_efficiency_RAC(q_RAC_h['q_rtd_c'], '（は）')
        d['heating']['sub'] = {
            'type'   : 'room_air_conditioner',
            'q_rtd_h': q_RAC_h['q_rtd_h'], 
            'q_max_h': q_RAC_h['q_max_h'],
            'e_rtd_h': e_RAC_h['e_rtd_h'],
        }        
    else :
        raise ValueError   
    
    return d

#### Example

In [34]:
d_web = {
    'Common':{
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 30.0,
        'TotalFloorArea': 90.0,
        'HouseType': 'detached'
    },
    'CoolingAndHeating':{
        'main' : {
            'cooling':{
                'type': 'convection',
                'division': '（い）'
            },
            'heating':{
                'type': 'convection',
                'division': '（ろ）'
            },
        },
        'other': {
            'cooling':{
                'type': 'convection',
                'division': '（ろ）'
            },
            'heating':{
                'type': 'radiation',
                'r': 0.7
            },
        },
    }
}

print(convert(d_web['Common']['MainOccupantRoomFloorArea'], **d_web['CoolingAndHeating']['main']))
print(convert(d_web['Common']['OtherOccupantRoomFloorArea'], **d_web['CoolingAndHeating']['other']))

{'cooling': {'main': {'q_rtd_c': 5600, 'q_max_c': 5944.619999999999, 'e_rtd_c': 3.2432}, 'sub': {}}, 'heating': {'main': {'q_rtd_h': 6685.3, 'q_max_h': 10047.047813999998, 'e_rtd_h': 4.157264}, 'sub': {}}}
{'cooling': {'main': {'q_rtd_c': 5600, 'q_max_c': 5944.619999999999, 'e_rtd_c': 3.0576}, 'sub': {}}, 'heating': {'main': {'construct_area': 21.0}, 'sub': {'q_rtd_h': 6685.3, 'q_max_h': 10047.047813999998, 'e_rtd_h': 3.855424}}}
